# 第3回

## 前回までの補足

Python に不慣れなために苦労している方がかなりいるように思います。
特に class 周りの表記に問題がありそうです。
"self" の意味などについてちょっとだけ説明します。

## Bellman-Ford Algorithm

Bellman-Fordアルゴリズムは負の重みが存在しても最適解を返すアルゴリズムである。
前回の復習だが、Dijkstra法は負の重みを持つ枝があると最善解を返さないことがある。
Dijkstra法のどこが問題なのか見ておこう。

### 負の重みに対するDijkstra法の動作

以下の図にDijkstra法の動作を示す。
負の重みがあると、ゴール節点の探索が終了した時点では解が誤っていることが分かる。
これは、節点cから節点aへ行く経路が見逃されているからだ。

<img src="dijkstra.png" width=600>

Bellman-Fordアルゴリズムはこのような場合にも正しい解を返すアルゴリズムである。
アルゴリズムとしてはDijkstra法よりも単純である。

### 単純な Bellman-Ford の疑似コード

まず単純なバージョンの疑似コードを示す。
メインのループを (節点数 - 1) ステップ実行する。
各ステップで、全ての節点の全ての枝をたどり最短距離の表を更新する。
もし負のサイクルがなければこの時点で最短距離が得られている。
また、(節点数 - 1) ステップの実行が終わってもまだ最短距離が更新されるなら
負の重みのサイクルがあるということだ。
計算量は非常に単純で、節点数 $V$ 総枝数 $E$ としたときに$ O(VE) $ である。

```
引数
prob: graph, start, goal
返値
最短経路のリスト、または負の重みが存在することを示す値

fun BellmanFordSimple (prob, f) {
  distance <- 各節点の距離のリスト。スタート節点は0、それ以外は無限大に初期化
  parent <- 各節点の親節点のリスト。全て-1などに初期化。

  # このループで全ての節点への距離を求める
  for 節点数 - 1 回 {
    for each node u {
      for each edge (u, v) with weight w {
        if distance[u] + w < distance[v] {
          distance[v] <- distance[u] + w
          parent[v] = u
        }
      }
    }
  }
  
  # ここで負のサイクルの存在を検出
  for each edge (u, v) with weight w {
    if distance[u] + w < distance[v] {
      return NEG_SYCLE
    }
  }

  return distance
```

### 現実的な Bellman-Ford の疑似コード

上の擬似コードは想像すれば分かるとおり非常に無駄が多い。
当然だが、最短距離が更新された節点からだけ枝をたどれば良いので
現実的には以下の様な実装をする。
計算量はグラフの形状による。
最悪の場合は上記と同様に $ O(VE) $ だが、
節点の次数 (節点から出る枝数) が少ない場合にはかなり高速に動作する。
この疑似コードの動作を以下の画像で示す。

```
引数
prob: graph, start, goal
返値
最短経路のリスト、または負の重みが存在することを示す値

fun BellmanFord (prob, f) {
  distance <- 各節点の距離のリスト。スタート節点は0、それ以外は無限大に初期化
  updated <- 各節点の距離が更新されたかどうかを示す。スタート節点のみtrue, 他はfalseに初期化
  parent <- 各節点の親節点のリスト。全て-1などに初期化。

  # このループで全ての節点への距離を求める
  for 節点数-1 回 {
    for each node u {
      if updated[u] {
        for each edge (u, v) with weight w {
          if distance[u] + w < distance[v] {
            distance[v] <- distance[u] + w
            parent[v] = u
            updated[v] = true
          }
        }
        updated[u] = false
      }
    }
    
    # updated が全て false ならその時点で終了
    if updated is false for all nodes {
      return distance
    }
  }
  
  # ここに到達したら負の経路が存在する
  return NEG_SYCLE
```

<img src="bellman_ford.gif" width=800>


## 情報あり探索 (Informed Search)

### A*探索と許容的なヒューリスティック (復習)

前回、A*法を紹介したが、つまりヒューリスティック関数$h(n)$が定義された場合の探索についてもう少し詳しく見てみよう。
2Dグリッド上の最短経路探索の場合のヒューリスティック関数について、前回は天下り的にマンハッタン距離を使う様に指示をした。
また、ヒューリスティック関数について、許容的かどうか (admissibleかどうか) が重要であることを紹介した。
復習だが、
節点$n$からゴールまでの正しい最短距離を $h^*(n)$ として、$h(n)$がどの節点についても$h^*(n)$を超えない場合に
$h(n)$を **許容的なヒューリスティック (admissible heuristic)** と言い
その場合にA*探索は最善解を返すと説明したのだった。

<img src="astar_fail.png" width=300>

逆にadmissibleでない場合にどのように失敗するのかを図示した。
大変単純な例だが、A*探索が失敗することは明らかだろう。
なお、ゴール節点で $h=0$ であるのは問題ない。ゴールからゴールへの最短距離は$0$だから、
許容的 (admissible) なヒューリスティック関数はゴールでは必ず$0$を返す。
ヒューリスティック探索が admissible であることはA*探索が最善解を返すための必要条件である。

ではこれだけで十分なのだろうか？実は違う。
以下の疑似コードが示すA*探索アルゴリズムについては、さらに必要な条件がある
（前回の BestFirstSearch の疑似コードと全く同じ)。

```
fun AStarSearch (prob, f) {
  open_list <- start 節点のみを含む priority queue, 関数fでソートされている
  closed_list <- start 節点のみを含む hash table
  while (open_list is not empty) {
    node <- Pop(open_list)
    if (prob.goal == node) return node
    for each child in Expand(prob, node) {
      st <- child.state
      if (st is not in closed_list or child.dist < closed_list[st].dist) {
        closed_list[s] <- child
        open_list.add(child)
      }
  return failure
```

### 単調性 (Consistency)

ヒューリスティック探索に必要なもう一つの性質は **単調性 (Consistency)** である。
下図の上の場合は単調ではなく、下は単調である。

<img src="consistency.png" width=400>

節点AとCについて、AからCの距離 (またはコスト) を $c(A, C)$ とする。
また、ゴール節点を$G$とする。
そのとき、単調なヒューリスティック関数 $h$ は以下の性質を満たす。  
$ h(A) - h(C) \leq c(A,C) $ かつ $h(G)=0$  
左側の式は以下の様に**三角不等式**のようにも書ける。  
$ h(A) \leq c(A,C) + h(C) $


重要な性質として、**単調な$h$は必ず許容的でもある**。
その逆は必ずしも成り立たない。
また、注目すべき点として$h$が単調なら$f=g+h$である場合
経路上の$f$の値は減少することはない。つまり以下が成り立つ。

$ h(A) \leq c(A, C) + h(C) => g(A) + h(A) \leq g(A) + c(A, C) + h(C) $

逆に言うと、もし許容的だが単調ではないヒューリスティック関数を使うと、
経路上の$f$が減少することがあるということだ。
これはDijkstra法が失敗する条件である負の重みがある場合と似ていないだろうか。
その場合、上記のA*探索は最善解を返す保証がない。

ここで考えてみよう。
上記のA*探索はDijkstra法とヒューリスティック関数を組み合わせたものだった。
では、Bellman-Fordアルゴリズムとヒューリスティック関数を組み合わせたらどうだろうか？
これは実際に可能であり、許容的だが単調でないヒューリスティック関数で最善解を得る方法の一つである。


### weighted A*探索と anytime アルゴリズム

A*探索の場合には節点からの距離を$g(n)$として、関数$f(n)=g(n)+h(n)$が小さい節点から順番に探索するのだった。
ここで、$f$の定義を変更して探索アルゴリズムの振る舞いを変更することができる。

- A* search: $f(n)=g(n)+h(n)$ とする探索
- weighted A* search (wA*): 重み $w$ を定義し、 $f(n)=g(n)+ w * h(n)$ とする探索
- greedy best first search: $f(n)=h(n)$ とする探索

wA*探索の利点は、重みを大きくするとA*探索よりも遙かに高速であることである。
ただし、$w*h(n)$はもちろん許容的ではないので、最適解の保証はない。
しかし全く保証がないわけではなく、weighted A*探索が発見する解は、最適解のコスト $f*$ の $w$ 倍以下であるという性質がある（証明略）。

greedy best first search は weighted A* 探索の重み $w$ を無限大にした物とも言える。
このアルゴリズムは多くの場合に非常に高速になんらかの解を発見する。

A*探索の一つの問題点は、探索にかかる時間が事前に分からないということだ。
時間制限があるときには、探索にかけた時間に応じて解が良くなっていく、
つまりいつ実行を止めてもその時点で得られる最善の解を返すような方法が望ましい。
これを anytime アルゴリズムという。

A*探索に anytime 性を持たせたい場合に、大きい $w$値から始めて wA* を実行し
徐々に $w$値を小さくしていくことで徐々に解の質を向上していくという方法が考えられる。
この手法は Anytime Reparing A* と呼ばれる。

> Likhachev, M., Gordon, G.J., Thrun, S.: ARA*: Anytime A* with provable bounds on sub-optimality. In: Advances in neural information processing systems, pp. 767–774 (2004)


## 第3回 課題:

- 以下によく似た一本道の簡単なグラフが2個ある。aからeへの採点経路をBellman-Fordアルゴリズムで求める場合、それぞれ何ステップかかるか。また結果を考察せよ。ただし、アルファベット順に探索をするものとする。

<img src="two_graphs.png" width=300>

- 前回の課題の解答で、展開の回数を比較していた人が居たが、これは大変良い。
    他によくある方法としては **search contour** を表示させるのも良い。
    これは探索終了時の open list と closed list のことである。テキストで良いので表示し、
    Dijkstra法, greedy best first search, A*, weighted A* で search contour を表示せよ。
    また違いを観察せよ。weighted A* search の重みはA*と振る舞いが変わるように適当に設定せよ。

- Bellman-ford アルゴリズムを実装し、以下のコード例中に示す2つのグラフについて最短経路探索を行え。
    両方の場合に正しい解を出すこと。コード例は使っても使わなくても良い。
    1. 負の重みを含むが負のサイクルがない場合 (以下のgraph_array)
    1. 負のサイクルがある場合 (以下のgraph_array2)
    
### 参考
- (外部リンク)
  AizuOnlineJudge の以下の問題を解いてみる。  
  single source shortest path (negative edges)
https://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=GRL_1_B&lang=ja


# 課題1
1. step = 4   ([a,b,c,d,e])
distance: [0,1000,1000,1000,1000] -> [0,-3,1000,1000,1000] -> [0,-3,-2,1000,1000] -> [0,-3,-2,-1,1000] -> [0,-3,-2,-1,0]

2. step = 4   ([a,b,c,d,e])
distance: [1000,1000,1000,1000,0] -> [1000,1000,1000,1,0] -> [1000,1000,2,1,0] -> [1000,3,2,1,0] -> [0,3,2,1,0]

# 課題2  

In [ ]:
# Dijkstra:  
# ********************  
# *000000000000000000*  
# *000000000000000000*  
# *00000S000000000000*  
# *000000000000000000*  
# *000000000000000000*  
# *000000000000*00000*  
# *000000000000*00000*  
# *000000000000*00000*  
# *000000000000*00000*  
# *000000000000*00000*  
# *00***********0000 *  
# *00000000   00000  *  
# *0000000    G000   *  
# *000000       0    *  
# *00000             *  
# *0000              *  
# *000               *  
# *00                *  
# ********************
  
# greedy best first search:  
# ********************
# *                  *
# *     0000000      *
# *    0S0000000     *
# *     000000000    *
# *        0000000   *
# *       00000*00   *
# *      000000*00   *
# *     0000000*00   *
# *     0000000*00   *
# *    00000000*00   *
# *  ***********00   *
# *           0000   *
# *           G00    *
# *                  *
# *                  *
# *                  *
# *                  *
# *                  *
# ********************

# A*(manhattan,w=1):
# ********************
# *    00000000      *
# *   0000000000     *
# *  000S00000000    *
# *  0000000000000   *
# *  0000000000000   *
# *  0000000000*00   *
# *  0000000000*00   *
# *  0000000000*00   *
# *  0000000000*00   *
# *  0000000000*00   *
# *  ***********00   *
# *           0000   *
# *           G00    *
# *                  *
# *                  *
# *                  *
# *                  *
# *                  *
# ********************


# weighted A*(manhattan, w=0.5):
# ********************
# *0000000000000000  *
# *00000000000000000 *
# *00000S000000000000*
# *00000000000000000 *
# *00000000000000000 *
# *000000000000*0000 *
# *000000000000*000  *
# *000000000000*000  *
# *000000000000*000  *
# *000000000000*000  *
# *00***********00   *
# *0000        000   *
# * 00        G000   *
# *             0    *
# *                  *
# *                  *
# *                  *
# *                  *
# ********************

# weighted A*(manhattan, w=2):
# ********************
# *          00      *
# *     00000000     *
# *    0S00000000    *
# *     000000000    *
# *      000000000   *
# *      000000*00   *
# *     0000000*00   *
# *     0000000*00   *
# *    00000000*00   *
# *    00000000*00   *
# *  ***********00   *
# *           0000   *
# *           G00    *
# *                  *
# *                  *
# *                  *
# *                  *
# *                  *
# ********************

# weighted A*(manhattan, w=2):
# ********************
# *                  *
# *     0000000      *
# *    0S0000000     *
# *     000000000    *
# *         000000   *
# *        0000*00   *
# *       00000*00   *
# *      000000*00   *
# *     0000000*00   *
# *    00000000*00   *
# *  ***********00   *
# *           0000   *
# *           G00    *
# *                  *
# *                  *
# *                  *
# *                  *
# *                  *
# ********************


weighted A*(large w):greedy best first search   
weighted A*(small w):Dijkstra

# 課題3

In [16]:
# 以下の1行はデバッグをするときのために入れてあります。
from pdb import set_trace as st
import numpy as np

# 負の閉路は含まないグラフ
graph_array = np.array(
#     [S, a, b, c, d, G]
    [[ 0, 3, 0, 4, 0, 0], # S
     [ 0, 0, 1, 0, 0, 0], # a
     [ 0, 0, 0, 0,-4, 3], # b
     [ 0,-2, 3, 0, 4, 0], # c
     [ 0, 0, 0, 0, 0, 1], # d
     [ 0, 0, 0, 0, 0, 0]] # G
)

# 負の閉路を含むグラフ
graph_array2 = np.array(
#     [S, a, b, c, d, G]
    [[ 0, 3, 0, 4, 0, 0], # S
     [ 0, 0, 1, 0, 0, 0], # a
     [ 0, 0, 0, 0,-4, 3], # b
     [ 0,-2, 3, 0, 4, 0], # c
     [ 0, 2, 0, 0, 0, 1], # d
     [ 0, 0, 0, 0, 0, 0]] # G
)

class NegativeWeightSSSP:
    def __init__(self, s, g, graph_array):
        self.start = s # node S
        self.goal = g # node G
        self.graph = graph_array

        
class BellmanFord:
    def __init__(self, problem):
        self.prob = problem
        self.node_num = self.prob.graph.shape[0]
  
    def expand(self, node):
        # return list of child nodes
        child_nodes = []
        for c in range(0, self.node_num):
            d = self.prob.graph[node][c]
            if d != 0:
                child_nodes.append(c)
        return child_nodes
    
    def search(self):
        start_node = self.prob.start

        distance = [10000] * self.node_num
        parent = [-1] * self.node_num
        updated = [False] * self.node_num

        distance[start_node] = 0
        parent[start_node] = -1
        updated[start_node] = True

        for step in range(0, self.node_num - 1):
            '''
            # 以下の1行を削除し、ここに実装すること
            '''
            for j in range(self.node_num):
                if updated[j]:
                    child_nodes=self.expand(j)
                    for i in child_nodes:
                        if distance[i]>distance[j]+self.prob.graph[j][i]:
                            distance[i]=distance[j]+self.prob.graph[j][i]
                            parent[i]=j
                            updated[i]=True
                    updated[j]=False
                
        if sum(updated)==0:
            return distance
        
        # None が返ったら負のサイクルがあったということ
        return None


prob1 = NegativeWeightSSSP(0, 5, graph_array)
solver = BellmanFord(prob1)
dist_list = solver.search()
print(dist_list)

print('------------------------')
prob2 = NegativeWeightSSSP(0, 5, graph_array2)
solver = BellmanFord(prob2)
dist_list = solver.search()
print(dist_list)

[0, 2, 3, 4, -1, 0]
------------------------
None


参考文献
- "Artificial Intelligence: A Modern Approach, 4th Global ed.", by Stuart Russell and Peter Norvig  
   http://aima.cs.berkeley.edu/index.html
- "ヒューリスティック探索入門", 陣内 佑  
   https://jinnaiyuu.github.io/pdf/textbook.pdf